# Imports

In [1]:
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import os
import requests
from IPython.core.debugger import set_trace
import pandas as pd

# Access Uniprot

## Retrieve Entries from Uniprot

In [11]:
url = 'https://www.uniprot.org/uploadlists/'
name_string = 'Q9XIE6 P98204 Q9C9Y1 Q8RX77 Q3E8X3 Q9LTW0 Q9M817 Q9LV10'

params = {
'from': 'ACC+ID',
'to': 'ACC',
'format': 'xml',
'query': ' '.join(name_string.split('\w'))
}

response = ''
data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
print('Starting batch entry retrieval.')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
    response = f.read().decode('utf-8')
print('finished')

Starting batch entry retrieval.
finished


## Extract Information of Entries

In [12]:
path = './Data/GO_Enrichment/RESULT_{}.txt'.format('FL_TransportLimited')

In [13]:
outfile = open(path, 'a')
accession_list = []
print('Creating Soup...')
FullSoup = BeautifulSoup(response, 'xml')
print('Soup Finished. Starting Parsing of Entries...')

for e in FullSoup.find_all('entry'):
    accession = e.find('accession').text
    uniprot_name = e.find('name').text
    accession_list.append(accession)
    
    print('Creating summary for : {}'.format(accession))
    function = e.find('comment', type="function")
    function = function.find('text').text if function != None else 'No function field found.'
    
    res_list = ['{} ({}, {})\n{}\n{}\nGO-terms :\n'.format(e.find(type="STRING")['id'], accession, uniprot_name, e.protein.fullName.text, function)]
    
    for p in e.findAll(type="GO"):
        res_list.append('{}\n'.format(p.property['value']))
    res_list.append('\n')
    
    outfile.write(''.join(res_list))
        
outfile.close()

print('\n' + ' '.join(accession_list))

Creating Soup...
Soup Finished. Starting Parsing of Entries...
Creating summary for : Q9XIE6
Creating summary for : P98204
Creating summary for : Q9C9Y1
Creating summary for : Q8RX77
Creating summary for : Q3E8X3
Creating summary for : Q9LTW0
Creating summary for : Q9M817
Creating summary for : Q9LV10

Q9XIE6 P98204 Q9C9Y1 Q8RX77 Q3E8X3 Q9LTW0 Q9M817 Q9LV10
